In [3]:
# Read csv input
# Filter required attributes
# Data Preprocessing
# Data Transformations
# Model Building
# Results Evaluation

In [4]:
import sys
print(sys.executable)

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow_new\python.exe


In [5]:
import pandas as pd
import numpy as np
import re
# import gensim
from sklearn.feature_extraction.text import TfidfVectorizer


In [6]:

filename_1=r'C:\\Raghu\\Work\\AI\\Agents_data\\Agent_labelled_input_withCRA_AS_OL_EXGSHO.csv'


input_df = pd.read_csv(filename_1, index_col=None, header=0)
input_df = input_df[['Troublshooting', 'Agent Description','Category_Classified']]
print(input_df.shape)
print(input_df.columns)


(8256, 3)
Index(['Troublshooting', 'Agent Description', 'Category_Classified'], dtype='object')


In [7]:
### Agent_labelled_input_withCRA_AS_OL_EXGSHO.csv

# This data set has CRA labelled as Online Returns
# This data set as well includes EXG & SHO


In [8]:
input_df.shape

(8256, 3)

In [9]:
input_df.columns

Index(['Troublshooting', 'Agent Description', 'Category_Classified'], dtype='object')

In [10]:
input_df['Category_Classified'].value_counts()

Online Status        2158
Online Return        1671
EXG/SHO              1305
General Query         729
Logistics Issues      669
Order Cancel          661
Refund Status         528
MCR                   304
Gift Card Queries     231
Name: Category_Classified, dtype: int64

In [11]:
print(input_df.shape)

(8256, 3)


In [12]:
# S similar comment with different categories doesn't make sense. We dont know what hold, so dropping all of the duplicate requests.
input_df = input_df.drop_duplicates(subset=['Troublshooting','Category_Classified'],keep=False)
print(input_df.shape)

(7207, 3)


In [13]:
# Identify Columns for building Classification Model
# From Business Understanding, Predictor Variable : Troubleshooting Comments  Target Variable : Agent Description
filtered_df = input_df[input_df['Troublshooting'].notnull()]
filtered_df = filtered_df.drop_duplicates(subset=['Troublshooting'], keep=False)
print('********')
print(filtered_df.shape)
print(filtered_df['Category_Classified'].value_counts())

********
(6310, 3)
Online Status        1927
Online Return        1090
EXG/SHO               709
General Query         615
Order Cancel          582
Logistics Issues      572
Refund Status         415
MCR                   228
Gift Card Queries     172
Name: Category_Classified, dtype: int64


### Data Preprocessing

In [14]:
# import spacy
# from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [15]:
import unicodedata
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import pickle
import nltk
%matplotlib inline



In [16]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


In [17]:
# Data Preprocessing
# Function to remove Fusion links from Troubleshooting comments
# Considering both Consumer and Commercial - Can't seggregate at the moment because we have an issue at the source in report query

def clear_fusion_links(text):
    start = re.escape("Fusion")
    end  = re.escape("*")
    try:
        match_text = re.search('%s(.*)%s' % (start, end), text).group(1)
        if match_text:
            result = start + match_text + '*'
            filtered_result = text.replace(result, '')
    except:
        filtered_result = text
        
    return filtered_result

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


def clean_text(text):   
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = re.sub(' +',' ', text)
    text = re.sub('xxx*', ' ', text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\r", "", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

custom_stop_words = ['price','email','custom','batteries','battery','batteri','mous','mouse','keyboard','product','order','laptop','screen','monitor','hello', 'regard', 'madam', 'website', 'dell', 'order', 'where', 'ought', 'doing', 'my', 'back', 'itself', 'him', 'its', 
                     'been', 'however', 'yours', 'whether', 'there', 'during', 'those', 'her', 'would', 'the', 'who', 'no', 'make', 'which', 'too', 
                     'more', 'here', 'yourselves', 'again', 'while', 'two', 'said', 'us', 'we', 'because', 'with', 'old', 'be', 'another', 'out', 
                     'second', 'having', 'made', 'ever', 'should', 'long', 'herself', 'our', 'few', 'does', 'she', 'it', 'being', 'when', 'ours', 
                     'are', 'many', 'could', 'had', 'same', 'any', 'go', 'until', 'if', 'then', 'just', 'now', 'first', 'his', 'these', 'me', 'get', 
                     'hers', 'against', 'were', 'to', 'further', 'one', 'after', 'very', 'off', 'was', 'like', 'theirs', 'says', 'three', 'new', 'once', 
                     'but', 'yourself', 'still', 'you', 'from', 'your', 'whom', 'up', 'and', 'how', 'before', 'about', 'above', 'other', 
                     'am', 'do', 'since', 'than', 'high', 'well', 'only', 'he', 'did', 'or', 'has', 'that', 'their', 'myself', 'into', 
                     'seen', 'every', 'some', 'both', 'way', 'over', 'this', 'own', 'between', 'less', 'by', 'a', 'i', 'of', 'themselves', 
                     'in', 'so', 'at', 'below', 'put', 'each', 'an', 'ourselves', 'five', 'as', 'such', 'through', 'for', 'take', 'also', 
                     'himself', 'even', 'see', 'them', 'is', 'have', 'what', 'goes', 'on', 'most', 'why', 'they', 'least', 'down', 'all', 'say', 
                     'under', 'four']

def preprocess(text):
    result=[]
    for token in text.split(' '):
        if token not in custom_stop_words and len(token)>1:
            result.append(token)
    return ' '.join(result)


# def simple_stemmer(text):
#     ps = nltk.porter.PorterStemmer()
#     text = ' '.join([ps.stem(word) for word in text.split()])
#     return text

In [18]:
print(len(filtered_df['Troublshooting']))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(lambda com : expand_contractions(str(com)))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(lambda com : clear_fusion_links(str(com)))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(lambda com : lemmatize_text(str(com)))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(lambda com : clean_text(str(com)))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(lambda com : remove_accented_chars(str(com)))
print(sum(filtered_df['Troublshooting'].str.len()))
filtered_df['Troublshooting'] = filtered_df['Troublshooting'].map(preprocess)
print(sum(filtered_df['Troublshooting'].str.len()))



6310
1396068
1398362
1367124
1356075
1338001
1337732
716065


In [19]:
filtered_df.shape

(6310, 3)

In [20]:
filtered_df.head(10)

,Troublshooting,Agent Description,Category_Classified
0,month ago set today think something resolution...,Online Return,Online Return
1,believe purchase model 9380 9370 become aware ...,Online Cancel,Order Cancel
2,offer choose color never ask color able change...,Online Other,General Query
3,cancel may re purchase can please cancel confi...,Online Cancel,Order Cancel
4,want computer backlit computer accidentally wi...,Online Return,Online Return
5,please confirm cancel thank kind john 07901738145,Online Cancel,Order Cancel
6,notice item return not know need full refund,Online Status,Online Return
7,hi wonder can return charger buy seem problem ...,Online Return,Online Return
8,system replacement customer not happy system c...,Online Other,Online Return
9,can cancel account paypal never refund charge ...,Online Status,Refund Status


In [21]:
# Check for Missing Values

percent_missing = filtered_df.isnull().sum() * 100 / len(filtered_df)
missing_value_ccn_df = pd.DataFrame({'column_name': filtered_df.columns,
                                 'percent_missing': percent_missing})
missing_value_ccn_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_ccn_df.head(10)

,column_name,percent_missing
Troublshooting,Troublshooting,0.0
Agent Description,Agent Description,0.0
Category_Classified,Category_Classified,0.0


In [22]:
# Create a dataframe which is copy of original dataframe
df_till_here = filtered_df

In [23]:
filtered_df = filtered_df.drop_duplicates()
filtered_df.shape

(6250, 3)

In [24]:
class_prior_list = filtered_df['Category_Classified'].value_counts(normalize=True).tolist()
print(class_prior_list)


[0.30352, 0.17344, 0.11312, 0.09808, 0.0912, 0.09024, 0.0664, 0.03648, 0.02752]


In [25]:
filtered_df['classes'] = filtered_df['Category_Classified'].factorize()[0]
categ_id_df = filtered_df[['Category_Classified', 'classes']].drop_duplicates().sort_values('classes')
categ_to_id = dict(categ_id_df.values)
id_to_categ = dict(categ_id_df[['classes', 'Category_Classified']].values)
print ('categ_to_id dictionary : {}'.format(categ_to_id))
print ('id_to_categ dictionary : {}'.format(id_to_categ))

categ_to_id dictionary : {'Online Return': 0, 'Order Cancel': 1, 'General Query': 2, 'Refund Status': 3, 'Logistics Issues': 4, 'Online Status': 5, 'MCR': 6, 'Gift Card Queries': 7, 'EXG/SHO': 8}
id_to_categ dictionary : {0: 'Online Return', 1: 'Order Cancel', 2: 'General Query', 3: 'Refund Status', 4: 'Logistics Issues', 5: 'Online Status', 6: 'MCR', 7: 'Gift Card Queries', 8: 'EXG/SHO'}


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow_new\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
sm = SMOTE(sampling_strategy='auto', k_neighbors=3, random_state=125)

In [27]:
# Train Test Split

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=42)

X = filtered_df['Troublshooting']
y = filtered_df['Category_Classified']

# print(y.value_counts())
       
for train_index, test_index in sss.split(X, y):
    X_train = X.iloc[train_index]
    X_test =  X.iloc[test_index]
    y_train = y.iloc[train_index] 
    y_test = y.iloc[test_index]
    
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print('###')
print(y_train.value_counts())
print('###')
print(y_test.value_counts())
print('###')

(4687,)
(1563,)
(4687,)
(1563,)
###
Online Status        1423
Online Return         813
EXG/SHO               530
General Query         460
Logistics Issues      427
Order Cancel          423
Refund Status         311
MCR                   171
Gift Card Queries     129
Name: Category_Classified, dtype: int64
###
Online Status        474
Online Return        271
EXG/SHO              177
General Query        153
Logistics Issues     143
Order Cancel         141
Refund Status        104
MCR                   57
Gift Card Queries     43
Name: Category_Classified, dtype: int64
###


In [28]:
# Vectorizing Text

# cv_mnb = CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)

tfidf = TfidfVectorizer(min_df=0.001,sublinear_tf=True, 
                            norm='l2',
                            analyzer='word', 
                            token_pattern=r'\w{1,}', 
                            encoding='latin-1',
                            ngram_range=(1, 3), 
                            stop_words='english',
                            lowercase=True)


# Term-Document Matrix
X_train_counts = tfidf.fit_transform(X_train)

# Transform a count matrix to a normalized tf or tf-idf representation
tfidf_transformer = TfidfTransformer(use_idf=False)

# # Fit to data and transform it
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [29]:
### Preparing balanced data set - handling class imbalance

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train shape is:',X_train.shape)
print('y_train shape is:',y_train.shape)
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)

X_train shape is: (4687,)
y_train shape is: (4687,)
X_train_smote shape is: (12807, 2905)
y_train_smote shape is: (12807,)


In [34]:
clf_mnb = MultinomialNB(alpha=0.3,class_prior=class_prior_list)

# calibrate model on validation data
calibrator_clf_mnb = CalibratedClassifierCV(clf_mnb, cv=10)
calibrator_clf_mnb.fit(X_train_smote, y_train_smote)

# # Saving Model
filename_mnb = 'MNB-experiment-10APR.sav'
with open(filename_mnb, 'wb') as fout:
    pickle.dump((tfidf, tfidf_transformer, calibrator_clf_mnb), fout)
    
# # Reading Model
with open(filename_mnb, 'rb') as f:
    tfidf, tfidf_transformer, Multinomial_NB_model = pickle.load(f)

class_dict_mnb = {}

for idx, val in enumerate(Multinomial_NB_model.classes_):
    class_dict_mnb[val] = idx
print(class_dict_mnb)


{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}


In [35]:
def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)


print('Train Accuracy Metrics')
ypred_train = Multinomial_NB_model.predict(tfidf.transform(X_train))
acc_mnb_train = accuracy_score(y_train, ypred_train)
roc_auc_mnb_train = multiclass_roc_auc_score(y_train, ypred_train)
print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc acc_mnb is:', acc_mnb_train)
print('roc_auc_mnb is:', roc_auc_mnb_train)
print('*****************')

# Validation on Test data
ypred = Multinomial_NB_model.predict(tfidf.transform(X_test))
acc_mnb_test = accuracy_score(y_test, ypred)
roc_auc_mnb_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_mnb_test)
print('Test ROC-AUC Score is:',roc_auc_mnb_test)
print('*****************')


Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.67      0.88      0.76       530
    General Query       0.56      0.78      0.65       460
Gift Card Queries       0.82      0.97      0.89       129
 Logistics Issues       0.79      0.87      0.83       427
              MCR       0.69      0.86      0.76       171
    Online Return       0.88      0.62      0.73       813
    Online Status       0.90      0.78      0.83      1423
     Order Cancel       0.78      0.80      0.79       423
    Refund Status       0.79      0.67      0.72       311

         accuracy                           0.78      4687
        macro avg       0.76      0.80      0.78      4687
     weighted avg       0.80      0.78      0.78      4687

*****************
train acc acc_mnb is: 0.7757627480264562
roc_auc_mnb is: 0.8880281681191771
*****************
                 
##################
Test Accuracy Metrics is:
##################
             

In [36]:
from sklearn.model_selection import cross_val_score  

scores_mnb = cross_val_score(Multinomial_NB_model, tfidf.transform(X_test), y_test, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_mnb.mean(), scores_mnb.std() * 2*100))

Accuracy: 0.61 (+/- 6.85)


In [32]:
# #GridSearch # MNB

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

tfidf_transformer = TfidfTransformer()
clf_mnb = MultinomialNB(class_prior=class_prior_list)


objs = [("vect", tfidf), 
        ("tfidf", tfidf_transformer),
        ("mnb", clf_mnb)]
pipe = Pipeline(objs)
pipe.named_steps
print(pipe.named_steps)

#

# Specify parameters of the pipeline and their ranges for grid search
params = {
    'vect__ngram_range':((1,2),(1,3)),
    'tfidf__use_idf': (True, False),
    'mnb__alpha': np.linspace(0.2, 1, 20)
}


# Construct our grid search object
search = GridSearchCV(pipe, param_grid=params, refit=False)
search.fit(X_train, y_train)

{'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='latin-1',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0.001, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None), 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), 'mnb': MultinomialNB(alpha=1.0,
              class_prior=[0.30352, 0.17344, 0.11312, 0.09808, 0.0912, 0.09024,
                           0.0664, 0.03648, 0.02752],
              fit_prior=True)}


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='latin-1',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=0.001,
                                                        ngram_range=(1, 3),
                                                 

In [33]:
print("Best parameter values:")
for param in search.best_params_.items():
    print(param)

Best parameter values:
('mnb__alpha', 0.24210526315789474)
('tfidf__use_idf', False)
('vect__ngram_range', (1, 3))


In [201]:
# cv_mnb = CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)
# cv_mnb = TfidfVectorizer(min_df=0.005,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
#                                 ngram_range=(1, 3), stop_words='english',lowercase=True)
# X_train_counts = cv_mnb.fit_transform(X_train)

# print(X_train_counts.size)

In [202]:
### Multinomial NaiveBayes


# Vectorizing Text

tfidf_mnb = TfidfVectorizer(min_df=0.001,sublinear_tf=True, 
                            norm='l2',
                            analyzer='word', 
                            token_pattern=r'\w{1,}', 
                            encoding='latin-1',
                            ngram_range=(1, 3), 
                            stop_words='english',
                            lowercase=True)


# Term-Document Matrix
X_train_counts = tfidf_mnb.fit_transform(X_train)

# Transform a count matrix to a normalized tf or tf-idf representation
tfidf_transformer = TfidfTransformer(use_idf=True)

# # Fit to data and transform it
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


# ### Preparing balanced data set - handling class imbalance

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)

### Model object creation and fitting

clf_mnb = MultinomialNB(alpha=0.5)
# clf_mnb.fit(X_train_smote, y_train_smote)

# # # #Test
# # # X_test_counts = cv_mnb.transform(X_test)
# # # tfidf_transformer = TfidfTransformer(use_idf=False)
# # # X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

# # calibrate model on validation data
calibrator_clf_mnb = CalibratedClassifierCV(clf_mnb, cv=10)
calibrator_clf_mnb.fit(X_train_smote, y_train_smote)

# # Saving Model
filename_mnb = 'MNB-experiment-30MAR.sav'
with open(filename_mnb, 'wb') as fout:
    pickle.dump((tfidf_mnb, tfidf_transformer, calibrator_clf_mnb), fout)
    
# # Reading Model
with open(filename_mnb, 'rb') as f:
    tfidf_mnb, tfidf_transformer, Multinomial_NB_model = pickle.load(f)

class_dict_mnb = {}

for idx, val in enumerate(Multinomial_NB_model.classes_):
    class_dict_mnb[val] = idx
print(class_dict_mnb)


X_train_smote shape is: (12807, 2914)
y_train_smote shape is: (12807,)
{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}


In [207]:
### Linear SVC

# count_vect_lsvc = CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)
tfidf_lsvc = TfidfVectorizer(min_df=0.001,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
                                ngram_range=(1, 3), stop_words='english',lowercase=True)

X_train_counts = tfidf_lsvc.fit_transform(X_train)
tfidf_transformer = TfidfTransformer(use_idf=True)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)

lin_svc = LinearSVC(C=0.01,multi_class='ovr',class_weight ='balanced',loss='squared_hinge',penalty='l2')
# lin_svc.fit(X_train_smote, y_train_smote)


# calibrate model on validation data
calibrator_lin_svc = CalibratedClassifierCV(lin_svc, cv=10)
calibrator_lin_svc.fit(X_train_smote, y_train_smote)

# Saving Model
filename_lsvc = 'C:\\Users\\raghu_chandra_katike\\notebooks\\AI-ML\\Linear_svc_lablled-30MAR.sav'
with open(filename_lsvc, 'wb') as fout:
    pickle.dump((tfidf_lsvc, tfidf_transformer, calibrator_lin_svc), fout)
    
# Reading the model
with open(filename_lsvc, 'rb') as f:
    tfidf_lsvc, tfidf_transformer_lsvc, model_lsvc = pickle.load(f)

class_dict_lsvc = {}

for idx, val in enumerate(model_lsvc.classes_):
    class_dict_lsvc[val] = idx
print(class_dict_lsvc)

X_train_smote shape is: (12807, 2914)
y_train_smote shape is: (12807,)
{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}


In [208]:
def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)

# Data Transoformation
# X_test = X_test.map(expand_contractions)
# X_test = X_test.map(clear_fusion_links)
# X_test = X_test.map(lemmatize_text)
# X_test = X_test.map(clean_text)
# X_test = X_test.map(remove_accented_chars)
# X_test = X_test.map(preprocess)

print('Train Accuracy Metrics')
ypred_train = model_lsvc.predict(tfidf_lsvc.transform(X_train))
acc_lsvc_train = accuracy_score(y_train, ypred_train)
roc_auc_lsvc_train = multiclass_roc_auc_score(y_train, ypred_train)
print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc acc_lsvc is:', acc_lsvc_train)
print('roc_auc_lsvc is:', roc_auc_lsvc_train)
print('*****************')

# Validation on Test data
ypred = model_lsvc.predict(tfidf_lsvc.transform(X_test))
acc_lsvc_test = accuracy_score(y_test, ypred)
roc_auc_lsvc_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_lsvc_test)
# print(metrics.confusion_matrix(y_test, ypred))
print('Test ROC-AUC Score is:',roc_auc_lsvc_test)
print('*****************')


Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.68      0.77      0.72       531
    General Query       0.61      0.54      0.57       460
Gift Card Queries       0.81      0.91      0.86       129
 Logistics Issues       0.72      0.82      0.77       427
              MCR       0.57      0.78      0.66       171
    Online Return       0.83      0.53      0.64       813
    Online Status       0.84      0.81      0.83      1423
     Order Cancel       0.67      0.85      0.75       423
    Refund Status       0.54      0.68      0.60       311

        micro avg       0.73      0.73      0.73      4688
        macro avg       0.70      0.74      0.71      4688
     weighted avg       0.74      0.73      0.73      4688

*****************
train acc acc_lsvc is: 0.7276023890784983
roc_auc_lsvc is: 0.8541121677389557
*****************
                 
##################
Test Accuracy Metrics is:
##################
           

In [209]:
from sklearn.model_selection import cross_val_score  

scores_lsvc = cross_val_score(model_lsvc,tfidf_lsvc.transform(X_test), y_test, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_lsvc.mean(), scores_lsvc.std() * 2))

Accuracy: 0.66 (+/- 0.06)


In [210]:
# #GridSearch

# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline

# count_vect_lsvc = CountVectorizer(encoding='latin-1',stop_words='english', lowercase=True)
# tfidf_transformer = TfidfTransformer()
# lin_svc = LinearSVC(class_weight ='balanced')


# objs = [("vect", count_vect_lsvc), 
#         ("tfidf", tfidf_transformer),
#         ("lin_svc", lin_svc)]
# pipe = Pipeline(objs)
# pipe.named_steps
# print(pipe.named_steps)

# #

# # Specify parameters of the pipeline and their ranges for grid search
# params = {
#     'vect__min_df':np.linspace(0,0.2,20),
#     'vect__max_df':np.linspace(0.3,1,20),
#     'vect__ngram_range':((1,2),(1,3)),
#     'lin_svc__C': np.linspace(0.2, 1, 10),
#     'lin_svc__multi_class':('ovr','crammer_singer')
# }



# # Construct our grid search object
# search = GridSearchCV(pipe, param_grid=params, refit=False)
# search.fit(X_train, y_train)

In [211]:
# print("Best parameter values:")
# for param in search.best_params_.items():
#     print(param)

In [212]:
# Bagging & Boosting Techniques




In [213]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier


# count_vect_rf = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)
tfidi_rf = TfidfVectorizer(min_df=0.001,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
                                ngram_range=(1, 3), stop_words='english',lowercase=True)

X_train_counts = tfidi_rf.fit_transform(X_train)
tfidf_transformer = TfidfTransformer(use_idf=True)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)

random_forest_model = RandomForestClassifier(n_estimators=300,max_depth=50,min_samples_leaf=5,min_samples_split=0.2,random_state=42,max_features='auto',class_weight='balanced').fit(X_train_tfidf, y_train)
# random_forest_model.fit(X_train_smote, y_train_smote)

# calibrate model on validation data
calibrator_rf_model = CalibratedClassifierCV(random_forest_model, cv=10)
calibrator_rf_model.fit(X_train_smote, y_train_smote)

# Saving Model
filename_rf = 'C:\\Users\\raghu_chandra_katike\\notebooks\\AI-ML\\Random-Forest-30MAR.sav'
with open(filename_rf, 'wb') as fout:
    pickle.dump((tfidi_rf, tfidf_transformer, calibrator_rf_model), fout)
    
# Reading the model
with open(filename_rf, 'rb') as f:
    tfidf_rf, tfidf_transformer_rf, random_forest_model = pickle.load(f)

class_dict_rf = {}

for idx, val in enumerate(random_forest_model.classes_):
    class_dict_rf[val] = idx
print(class_dict_rf)
print(calibrator_rf_model)

X_train_smote shape is: (12807, 2914)
y_train_smote shape is: (12807,)
{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}
CalibratedClassifierCV(base_estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=50, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=0.2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=None, oob_score=False,
            random_state=42, verbose=0, warm_start=False),
            cv=10, method='sigmoid')


In [214]:
# def multiclass_roc_auc_score(y_test, ypred, average="macro"):
#     lb = LabelBinarizer()
#     lb.fit(y_test)
#     y_test = lb.transform(y_test)
#     y_pred = lb.transform(ypred)
#     return roc_auc_score(y_test, y_pred, average=average)

# # Data Transoformation
# # X_test = X_test.map(expand_contractions)
# # X_test = X_test.map(clear_fusion_links)
# # X_test = X_test.map(lemmatize_text)
# # X_test = X_test.map(clean_text)
# # X_test = X_test.map(remove_accented_chars)
# # X_test = X_test.map(preprocess)


# # Validation on Test data
# ypred = random_forest_model.predict(tfidf_rf.transform(X_test))
# acc_rf = accuracy_score(y_test, ypred)
# roc_auc_rf = multiclass_roc_auc_score(y_test, ypred)

# # print ('ypred is:',ypred)
# print('*****************')
# print('Accuracy is:',acc_rf)
# print('*****************')
# print(metrics.classification_report(y_test, ypred))
# print('*****************')
# print(metrics.confusion_matrix(y_test, ypred))
# print('*****************')
# print('ROC-AUC Score is:',roc_auc_rf)

In [215]:
def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)

# Data Transoformation
# X_test = X_test.map(expand_contractions)
# X_test = X_test.map(clear_fusion_links)
# X_test = X_test.map(lemmatize_text)
# X_test = X_test.map(clean_text)
# X_test = X_test.map(remove_accented_chars)
# X_test = X_test.map(preprocess)

print('Train Accuracy Metrics')
ypred_train = random_forest_model.predict(tfidf_rf.transform(X_train))
acc_rf_train = accuracy_score(y_train, ypred_train)
roc_auc_rf_train = multiclass_roc_auc_score(y_train, ypred_train)
print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc acc_rf is:', acc_rf_train)
print('roc_auc_rf is:', roc_auc_rf_train)
print('*****************')

# Validation on Test data
ypred = random_forest_model.predict(tfidf_rf.transform(X_test))
acc_rf_test = accuracy_score(y_test, ypred)
roc_auc_rf_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_rf_test)
# print(metrics.confusion_matrix(y_test, ypred))
print('Test ROC-AUC Score is:',roc_auc_rf_test)
print('*****************')


Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.65      0.57      0.61       531
    General Query       0.30      0.55      0.39       460
Gift Card Queries       0.71      0.88      0.79       129
 Logistics Issues       0.70      0.78      0.74       427
              MCR       0.50      0.60      0.54       171
    Online Return       0.76      0.29      0.42       813
    Online Status       0.81      0.71      0.75      1423
     Order Cancel       0.62      0.83      0.71       423
    Refund Status       0.43      0.55      0.49       311

        micro avg       0.61      0.61      0.61      4688
        macro avg       0.61      0.64      0.60      4688
     weighted avg       0.67      0.61      0.61      4688

*****************
train acc acc_rf is: 0.6130546075085325
roc_auc_rf is: 0.7961109713670667
*****************
                 
##################
Test Accuracy Metrics is:
##################
               

In [216]:
from sklearn.model_selection import cross_val_score  

scores_rf = cross_val_score(random_forest_model, tfidf_lsvc.transform(X_test), y_test, cv=10)
print(scores_rf)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std() * 2))

[0.67901235 0.625      0.63522013 0.62420382 0.64102564 0.6516129
 0.6516129  0.63398693 0.66666667 0.64705882]
Accuracy: 0.65 (+/- 0.03)


In [217]:
# ## GridSearch
# # Random Forest

# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline

# count_vect_rf =  CountVectorizer(min_df=10,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)
# tfidf_transformer = TfidfTransformer(use_idf=False)
# random_forest_model = RandomForestClassifier(bootstrap=True,max_features='auto',oob_score=True,class_weight='balanced',random_state=42)


# objs = [("vect", count_vect_rf), 
#         ("tfidf", tfidf_transformer),
#         ("random_forest_model", random_forest_model)]
# pipe = Pipeline(objs)
# pipe.named_steps
# print(pipe.named_steps)

# #

# # Specify parameters of the pipeline and their ranges for grid search
# params = {
#     'random_forest_model__n_estimators': np.linspace(100, 500, 5,dtype=int),
#     'random_forest_model__max_depth':np.linspace(50, 300, 6,dtype=int),
#     'random_forest_model__min_samples_split':np.linspace(0.2, 1, 5),
#     'random_forest_model__min_samples_leaf':np.linspace(10, 50, 5,dtype=int)
# }



# # Construct our grid search object
# search = GridSearchCV(pipe, param_grid=params, refit=False)
# search.fit(X_train, y_train)

In [218]:
# print("Best parameter values:")
# for param in search.best_params_.items():
#     print(param)

In [219]:
# Logistic Regression
# from sklearn.pipeline import Pipeline

# cv_logistic = CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)

cv_logistic = TfidfVectorizer(min_df=0.005,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
                                ngram_range=(1, 3), stop_words='english',lowercase=True)
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_counts = cv_logistic.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf_logistic_model =  LogisticRegression(class_weight='balanced', max_iter=500, random_state=10, solver='lbfgs', multi_class='ovr')
# clf_logistic_model.fit(X_train_tfidf, y_train)

#Test
X_test_counts = cv_logistic.transform(X_test)
tfidf_transformer = TfidfTransformer(use_idf=False)
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)

# calibrate model on validation data
calibrator_clf_logistic = CalibratedClassifierCV(clf_logistic_model, cv=10)
calibrator_clf_logistic.fit(X_train_smote, y_train_smote)


# Saving Model
filename_logistic = 'Logistic-experiment-30MAR.sav'
with open(filename_logistic, 'wb') as fout:
    pickle.dump((cv_logistic, tfidf_transformer, calibrator_clf_logistic), fout)
    
# Reading Model
with open(filename_logistic, 'rb') as f:
    cv_logistic, tfidf_transformer, calibrator_clf_logistic = pickle.load(f)

class_dict_logistic = {}

for idx, val in enumerate(calibrator_clf_logistic.classes_):
    class_dict_logistic[val] = idx
print(class_dict_logistic)



X_train_smote shape is: (12807, 554)
y_train_smote shape is: (12807,)
{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}


In [220]:
def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)

print('Train Accuracy Metrics')
ypred_train = calibrator_clf_logistic.predict(cv_logistic.transform(X_train))
acc_logistic_train = accuracy_score(y_train, ypred_train)
roc_auc_logistic_train = multiclass_roc_auc_score(y_train, ypred_train)

print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc aacc_logistic_train is:', acc_logistic_train)
print('roc_auc_logistic_train is:', roc_auc_logistic_train)
print('*****************')

# Validation on Test data
ypred = calibrator_clf_logistic.predict(cv_logistic.transform(X_test))
acc_logistic_test = accuracy_score(y_test, ypred)
roc_auc_logistic_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_logistic_test)
print('Test ROC-AUC Score is:',roc_auc_logistic_test)
print('*****************')


Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.67      0.78      0.72       531
    General Query       0.61      0.60      0.60       460
Gift Card Queries       0.80      0.94      0.86       129
 Logistics Issues       0.75      0.87      0.81       427
              MCR       0.56      0.85      0.67       171
    Online Return       0.81      0.57      0.67       813
    Online Status       0.88      0.78      0.83      1423
     Order Cancel       0.74      0.84      0.79       423
    Refund Status       0.56      0.73      0.64       311

        micro avg       0.74      0.74      0.74      4688
        macro avg       0.71      0.77      0.73      4688
     weighted avg       0.76      0.74      0.74      4688

*****************
train acc aacc_logistic_train is: 0.7425341296928327
roc_auc_logistic_train is: 0.8702315756572813
*****************
                 
##################
Test Accuracy Metrics is:
#########

In [221]:
from sklearn.model_selection import cross_val_score  

scores_logistic = cross_val_score(calibrator_clf_logistic, X_test_tfidf, y_test, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_logistic.mean(), scores_logistic.std() * 2))

Accuracy: 0.65 (+/- 0.05)


In [229]:
# SGD Classifier

from sklearn.linear_model import SGDClassifier


# cv_sgd = CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)

cv_sgd = TfidfVectorizer(min_df=0.005,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
                                ngram_range=(1, 3), stop_words='english',lowercase=True)

tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_counts = cv_sgd.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
sgd_model =  SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=50, tol=None)
# sgd_model.fit(X_train_tfidf, y_train)

#Test
# X_test_counts = cv_sgd.transform(X_test)
# tfidf_transformer = TfidfTransformer(use_idf=False)
# X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)


# calibrate model on validation data
calibrator_sgd = CalibratedClassifierCV(sgd_model, cv=10)
calibrator_sgd.fit(X_train_smote, y_train_smote)


# Saving Model
filename_logistic = 'SGDClassifier-30MAR.sav'
with open(filename_logistic, 'wb') as fout:
    pickle.dump((cv_sgd, tfidf_transformer, calibrator_sgd), fout)
    
# Reading Model
with open(filename_logistic, 'rb') as f:
    cv_sgd, tfidf_transformer, calibrator_sgd = pickle.load(f)

class_dict_sgd = {}

for idx, val in enumerate(calibrator_sgd.classes_):
    class_dict_sgd[val] = idx
print(class_dict_sgd)



X_train_smote shape is: (12807, 554)
y_train_smote shape is: (12807,)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}


In [230]:
def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)

print('Train Accuracy Metrics')
ypred_train = calibrator_sgd.predict(cv_sgd.transform(X_train))
acc_sgd_train = accuracy_score(y_train, ypred_train)
roc_auc_sgd_train = multiclass_roc_auc_score(y_train, ypred_train)

print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc aacc_sgd_train is:', acc_sgd_train)
print('roc_auc_sgd_train is:', roc_auc_sgd_train)
print('*****************')

# Validation on Test data
ypred = calibrator_sgd.predict(cv_sgd.transform(X_test))
acc_sgd_test = accuracy_score(y_test, ypred)
roc_auc_sgd_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_sgd_test)
print('Test ROC-AUC Score is:',roc_auc_sgd_test)
print('*****************')


Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.63      0.72      0.67       531
    General Query       0.51      0.55      0.53       460
Gift Card Queries       0.78      0.90      0.84       129
 Logistics Issues       0.72      0.82      0.77       427
              MCR       0.46      0.77      0.57       171
    Online Return       0.75      0.54      0.63       813
    Online Status       0.86      0.73      0.79      1423
     Order Cancel       0.68      0.78      0.73       423
    Refund Status       0.56      0.67      0.61       311

        micro avg       0.70      0.70      0.70      4688
        macro avg       0.66      0.72      0.68      4688
     weighted avg       0.72      0.70      0.70      4688

*****************
train acc aacc_sgd_train is: 0.6953924914675768
roc_auc_sgd_train is: 0.8412127700777396
*****************
                 
##################
Test Accuracy Metrics is:
##################


In [224]:
from sklearn.model_selection import cross_val_score  

scores_sgd = cross_val_score(calibrator_sgd, X_test_tfidf, y_test, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_sgd.mean(), scores_sgd.std() * 2))

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

Accuracy: 0.64 (+/- 0.04)


In [225]:
from sklearn.ensemble import VotingClassifier

#
# count_vect =  CountVectorizer(min_df=0.005,analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), stop_words = 'english', lowercase=True)

count_vect = TfidfVectorizer(min_df=0.001,sublinear_tf=True, norm='l2',analyzer='word', token_pattern=r'\w{1,}', encoding='latin-1', 
                                ngram_range=(1, 3), stop_words='english',lowercase=True)

X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_smote, y_train_smote = sm.fit_resample(X_train_tfidf, y_train.ravel())
print('X_train_smote shape is:', X_train_smote.shape)
print('y_train_smote shape is:', y_train_smote.shape)


svc_model = LinearSVC(C=0.2,multi_class='ovr',class_weight ='balanced',loss='squared_hinge',penalty='l2')
# svc_model.fit(X_train_tfidf, y_train)
calibrator_svc_model = CalibratedClassifierCV(svc_model, cv=5, method='sigmoid')
calibrator_svc_model.fit(X_train_smote, y_train_smote)

mnb_model = MultinomialNB(alpha=0.5,class_prior=class_prior_list)
# mnb_model.fit(X_train_smote, y_train_smote)
calibrator_mnb_model = CalibratedClassifierCV(mnb_model, cv=5, method='sigmoid')
calibrator_mnb_model.fit(X_train_smote, y_train_smote)

random_forest_model = RandomForestClassifier(n_estimators=200,max_depth=50,min_samples_leaf=5,min_samples_split=0.2,random_state=42,max_features='auto',class_weight='balanced').fit(X_train_tfidf, y_train)
# random_forest_model.fit(X_train_smote, y_train_smote)
calibrator_rf_model = CalibratedClassifierCV(random_forest_model, cv=5, method='sigmoid')
calibrator_rf_model.fit(X_train_smote, y_train_smote)

clf_logistic_model =  LogisticRegression(class_weight='balanced', max_iter=500, random_state=10, solver='lbfgs', multi_class='ovr')
# clf_logistic_model.fit(X_train_smote, y_train_smote)
calibrator_log_model = CalibratedClassifierCV(clf_logistic_model, cv=5, method='sigmoid')
calibrator_log_model.fit(X_train_smote, y_train_smote)

sgd_model =  SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=50, tol=None)
# sgd_model.fit(X_train_smote, y_train_smote)
calibrator_sgd = CalibratedClassifierCV(sgd_model, cv=10)
calibrator_sgd.fit(X_train_smote, y_train_smote)

# voting_clf = VotingClassifier(estimators=[('mnb_model', calibrator_mnb_model), ('svc_model', calibrator_svc_model), 
#                                           ('random_forest_model', calibrator_rf_model),('clf_logistic_model', calibrator_log_model),
#                                          ('SGD_classifier_model',calibrator_sgd)], voting='hard',weights=[1,1,1,1,1])
voting_clf = VotingClassifier(estimators=[('mnb_model', calibrator_mnb_model), 
                                          ('random_forest_model', calibrator_rf_model),
                                          ('clf_logistic_model', calibrator_log_model),('SGD_classifier_model',calibrator_sgd),
                                          ('svc_model', calibrator_svc_model)], voting='hard',weights=[1,2,1,2,1])
voting_clf = voting_clf.fit(X_train_smote, y_train_smote)


# Model Saving & retreival

filename_rf = 'C:\\Users\\raghu_chandra_katike\\notebooks\\AI-ML\\Voting-Classifier-30MAR-MNB-RF-SVC-LOG.sav'
with open(filename_rf, 'wb') as fout:
    pickle.dump((count_vect, tfidf_transformer, voting_clf), fout)
    
# Reading the model
with open(filename_rf, 'rb') as f:
    count_vect, tfidf_transformer, voting_clf = pickle.load(f)

class_dict_rf = {}

for idx, val in enumerate(random_forest_model.classes_):
    class_dict_rf[val] = idx
print(class_dict_rf)


# Validation

def multiclass_roc_auc_score(y_test, ypred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(ypred)
    return roc_auc_score(y_test, y_pred, average=average)

print('Train Accuracy Metrics')
ypred_train = voting_clf.predict(count_vect.transform(X_train))
acc_vclf_train = accuracy_score(y_train, ypred_train)
roc_auc_vclf_train = multiclass_roc_auc_score(y_train, ypred_train)
print(metrics.classification_report(y_train, ypred_train))
print('*****************')
print('train acc acc_rf is:', acc_vclf_train)
print('roc_auc_rf is:', roc_auc_vclf_train)
print('*****************')

# Validation on Test data
ypred = voting_clf.predict(count_vect.transform(X_test))
acc_vclf_test = accuracy_score(y_test, ypred)
roc_auc_vclf_test = multiclass_roc_auc_score(y_test, ypred)

print('                 ')
print('##################')
print ('Test Accuracy Metrics is:')
print('##################')
print(metrics.classification_report(y_test, ypred))
print('*****************')
print('Test Accuracy is:',acc_vclf_test)
# print(metrics.confusion_matrix(y_test, ypred))
print('Test ROC-AUC Score is:',roc_auc_vclf_test)
print('*****************')

X_train_smote shape is: (12807, 2914)
y_train_smote shape is: (12807,)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


{'EXG/SHO': 0, 'General Query': 1, 'Gift Card Queries': 2, 'Logistics Issues': 3, 'MCR': 4, 'Online Return': 5, 'Online Status': 6, 'Order Cancel': 7, 'Refund Status': 8}
Train Accuracy Metrics
                   precision    recall  f1-score   support

          EXG/SHO       0.74      0.86      0.79       531
    General Query       0.69      0.73      0.71       460
Gift Card Queries       0.86      0.97      0.91       129
 Logistics Issues       0.81      0.89      0.85       427
              MCR       0.68      0.91      0.78       171
    Online Return       0.90      0.68      0.78       813
    Online Status       0.91      0.83      0.87      1423
     Order Cancel       0.77      0.88      0.82       423
    Refund Status       0.71      0.75      0.72       311

        micro avg       0.81      0.81      0.81      4688
        macro avg       0.78      0.83      0.80      4688
     weighted avg       0.82      0.81      0.81      4688

*****************
train acc acc_rf i

In [226]:
from sklearn.model_selection import cross_val_score  

scores_vclf = cross_val_score(voting_clf, X_test_tfidf, y_test, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_vclf.mean(), scores_vclf.std() * 2))

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\lib\site-packages\skle

Accuracy: 0.65 (+/- 0.04)


In [231]:
# Custom Validation Inputs

corpus = []
mnb_label = []
lsvc_label = []
rf_label = []
log_label=[]
sgd_label=[]
vclf_label = []
mnb_prob = []
lsvc_prob = []
rf_prob=[]
log_prob=[]
sgd_prob=[]

unseen_documents = [
"""please cancel my order.""",
"""There is a crack on the screen""",
""" Hi,  The order says delivered, but i did not receive my order.""",
"""This order has never been received.  Could you please provide status update.  Tracking states it was shipped 04/24/2019, but it has not been received""",
"""It has said that the mouse pad that I ordered was shipped since 4-16  but it does not seem like it has been shipped yet.  The tracking for it just says that the shipping label has been created.  I am just trying to get the status on what is going on with it.  Thanks for your time, Harry"""
"""I would like to return this order. I found the power cord that was lost, so don't need this one.""",
"""Hi I would just like to cancel my order i placed cause i realized i can not financially afford it at this time  With thanks  Mike Bachynski""",
"""When I ordered this, I thought it was a power bank and that I could power up non-Dell products with it. After receiving it, I see now that I need a Dell type charger to charge up this battery pack, which is not what I wanted. I would like to purchase something else.""",
"""I have cancelled the order, why is the money not returned to me?""",
"""Hi - I would like to return the : Dell Premier Wireless Keyboard and Mouse-KM717 - UK (QWERTY)  As I am quite happy with the standard one supplied with the computer.""",
"""I requested a return and have not received an email with the shipping label.""",
"""This camera is very slow for what I am wanting to take pictures of. I want to return it for a different camera.""",
"""I put in a return order request a few days ago and never received the email with the return label and instructions. I submitted it again today and still have not received it.""",
"""I have already set up to return the item but I never received my shipping documents for the return. Order Number 524360426""",
"""Please send me a detailed copy of my order placed 5/12/2019 --- itemizing all components of order..Other confirming emails from Dell do not allow access to details of order.I spent $15 for speedy delivery""",
"""I want the package shipped to a different address or pick it up at a FedEx facility near me, but on the FedEx website it said that I'm not allowed to make any changes due to shipper's restrictions. So I need your help with this issue, Thanks Efrat
""",
"""This product the monitor, Track Number IZ61R8926801553477, has been dispatched from Tamworth to Bristol instead of being delivered to my home address. We have spent over an hour on the phone to UPS to no avail of giving an order status of when it is being sent back from Bristol  to be delivered . Please chase this order delivery status as a computer no good without a screen !
""",
"""I will be home on12/28 and 12/29. Can we change the delivery date for this order to one of those dates please? Its an expensive item and would prefer to spend the extra $ for delivery when I will be home to avoid porch pirates. I approve any extra delivery charges.
""",
"""Hi was checking on the status of pending Dell rewards as it has been over 30 days and they are still pending and haven't been added to the account yet. thanks
"""]

# unseen_documents = [
# """I ordered a non-touch screen laptop with a 3.8 GHz processor. I got a touch screen model with a 2 GHz processor.
# """]

for i in unseen_documents:
    
    print('##############')
    print('comment is:', i)
    unseen_document = expand_contractions(i)
    print(unseen_document)
    unseen_document = clear_fusion_links(unseen_document)
    print(unseen_document)
    unseen_document = lemmatize_text(unseen_document)
    print(unseen_document)
    unseen_document = clean_text(unseen_document)
    print(unseen_document)
    unseen_document = remove_accented_chars(unseen_document)
    print(unseen_document)
    corpus.append(unseen_document)
    unseen_document = ''.join(preprocess(unseen_document))
    print('post preprocessing:', unseen_document)
    print('*********')
    
    
    # Multinomial Naive Bayes Prediction
    print('MNB Label is: ',Multinomial_NB_model.predict(tfidf_mnb.transform([unseen_document]))[0])
    mnb_label.append(Multinomial_NB_model.predict_proba(tfidf_mnb.transform([unseen_document]))[0])
    index = class_dict_mnb[Multinomial_NB_model.predict(tfidf_mnb.transform([unseen_document]))[0]]
    print('MNB Probability is: ',Multinomial_NB_model.predict_proba(tfidf_mnb.transform([unseen_document]))[0][index])
    mnb_prob.append(Multinomial_NB_model.predict_proba(tfidf_mnb.transform([unseen_document]))[0][index])
    print('*********')
    
    # Linear SVC Prediction
    print('SVC Label is: ',model_lsvc.predict(tfidf_lsvc.transform([unseen_document]))[0])
    lsvc_label.append(model_lsvc.predict(tfidf_lsvc.transform([unseen_document]))[0])
    index = class_dict_lsvc[model_lsvc.predict(tfidf_lsvc.transform([unseen_document]))[0]]
    print('SVC Probability is:',model_lsvc.predict_proba(tfidf_lsvc.transform([unseen_document]))[0][index])
    lsvc_prob.append(model_lsvc.predict_proba(tfidf_lsvc.transform([unseen_document]))[0][index])
    print('*********')
    

    # Random Forest Classifier
    print('RF Label is: ',random_forest_model.predict(tfidi_rf.transform([unseen_document]))[0])
    rf_label.append(random_forest_model.predict(tfidi_rf.transform([unseen_document]))[0])
    index = class_dict_rf[random_forest_model.predict(tfidi_rf.transform([unseen_document]))[0]]
    print('RF Probability is:',random_forest_model.predict_proba(tfidi_rf.transform([unseen_document]))[0][index])
    rf_prob.append(random_forest_model.predict_proba(tfidi_rf.transform([unseen_document]))[0][index])
    print('*********')
    
    # Logistic Regression Classifier
    print('LR Label is: ',calibrator_clf_logistic.predict(cv_logistic.transform([unseen_document]))[0])
    log_label.append(calibrator_clf_logistic.predict(cv_logistic.transform([unseen_document]))[0])
    index = class_dict_logistic[calibrator_clf_logistic.predict(cv_logistic.transform([unseen_document]))[0]]
    print('LR Probability is: ',calibrator_clf_logistic.predict_proba(cv_logistic.transform([unseen_document]))[0][index])
    log_prob.append(calibrator_clf_logistic.predict_proba(cv_logistic.transform([unseen_document]))[0][index])
    print('*********')
    
    # SGD Classifier
    print('SGD Label is: ',calibrator_sgd.predict(cv_sgd.transform([unseen_document]))[0])
    sgd_label.append(calibrator_sgd.predict(cv_sgd.transform([unseen_document]))[0])
    index = class_dict_sgd[calibrator_sgd.predict(cv_sgd.transform([unseen_document]))[0]]
    print('SGD Probability is: ',calibrator_sgd.predict_proba(cv_sgd.transform([unseen_document]))[0][index])
    sgd_prob.append(calibrator_sgd.predict_proba(cv_sgd.transform([unseen_document]))[0][index])
    print('*********')
    
     # Voting Classifier
    print('VCLF Label is: ',voting_clf.predict(count_vect.transform([unseen_document]))[0])
    vclf_label.append(voting_clf.predict(count_vect.transform([unseen_document]))[0])
    print('*********')
    


##############
comment is: please cancel my order.
please cancel my order.
please cancel my order.
please cancel my order .
please cancel my order .
please cancel my order .
post preprocessing: please cancel
*********
MNB Label is:  Order Cancel
MNB Probability is:  0.8758416419164524
*********
SVC Label is:  Order Cancel
SVC Probability is: 0.9906246527523368
*********
RF Label is:  Order Cancel
RF Probability is: 0.4599737790871494
*********
LR Label is:  Order Cancel
LR Probability is:  0.9619123876839837
*********
SGD Label is:  Order Cancel
SGD Probability is:  0.9173080607322012
*********
VCLF Label is:  Order Cancel
*********
##############
comment is: There is a crack on the screen
There is a crack on the screen
There is a crack on the screen
there be a crack on the screen
there be a crack on the screen
there be a crack on the screen
post preprocessing: crack
*********
MNB Label is:  EXG/SHO
MNB Probability is:  0.8921733041069679
*********
SVC Label is:  EXG/SHO
SVC Probabilit

comment is: I have already set up to return the item but I never received my shipping documents for the return. Order Number 524360426
I have already set up to return the item but I never received my shipping documents for the return. Order Number 524360426
I have already set up to return the item but I never received my shipping documents for the return. Order Number 524360426
I have already set up to return the item but I never receive my shipping document for the return . order number 524360426
i have already set up to return the item but i never receive my shipping document for the return . order number 524360426
i have already set up to return the item but i never receive my shipping document for the return . order number 524360426
post preprocessing: already set return item never receive shipping document return number 524360426
*********
MNB Label is:  Online Return
MNB Probability is:  0.6304016434289644
*********
SVC Label is:  Online Return
SVC Probability is: 0.5149844512701

In [178]:
ResultsDf = pd.DataFrame({'Method':['MNB','SVC','RF','LR','SGD','VCLF'], 
                          'Train accuracy': [acc_mnb_train,acc_lsvc_train,acc_rf_train,acc_logistic_train,acc_sgd_train,acc_vclf_train],
                          'Train_ROC_AUC_Score':[roc_auc_mnb_train,roc_auc_lsvc_train,roc_auc_rf_train,roc_auc_logistic_train,roc_auc_sgd_train,roc_auc_vclf_train],
                          'Test accuracy': [acc_mnb_test,acc_lsvc_test,acc_rf_test,acc_logistic_test,acc_sgd_test,acc_vclf_test],
                          'TestROC_AUC_Score':[roc_auc_mnb_test,roc_auc_lsvc_test,roc_auc_rf_test,roc_auc_logistic_test,roc_auc_sgd_test,roc_auc_vclf_test],
                         'Cross_Validation_Score':[scores_mnb.mean(),scores_lsvc.mean(),scores_rf.mean(),scores_logistic.mean(),scores_sgd.mean(),scores_vclf.mean()]})
ResultsDf

,Method,Train accuracy,Train_ROC_AUC_Score,Test accuracy,TestROC_AUC_Score,Cross_Validation_Score
0,MNB,0.698260,0.794854,0.625000,0.739512,0.605217
1,SVC,0.774267,0.851545,0.646902,0.762929,0.647674
2,RF,0.650183,0.785133,0.614316,0.757704,0.623896
3,LR,0.739011,0.821337,0.656517,0.763940,0.647417
4,SGD,0.730769,0.810958,0.649573,0.756678,0.624983
5,VCLF,0.743819,0.827581,0.654915,0.764029,0.644206


In [120]:
total_tags = []


for sentence in corpus:
    tags=[]
    temp_entity_name = ''
    temp_named_entity = None
    sentence = nlp(sentence)
    for word in sentence.ents:
        term = word.text 
        tag = word.label_
        if tag:
            tags.append(tag)
    total_tags.append(tags)

ResultsDf = pd.DataFrame({'original_text':unseen_documents,'Sentence':corpus, 'Entities': total_tags, 'mnb_label':mnb_label,'lsvc_label':lsvc_label,'rf_label':rf_label,
                          'mnb_prob': mnb_prob,'lsvc_prob':lsvc_prob,'rf_prob':rf_prob
                         })
ResultsDf.to_csv('C:\\Raghu\\Work\\AI\\AI-Results-Validation\\SupervisedResultsAnalysis\\out03MAR.csv') 
ResultsDf.head(10)

,original_text,Sentence,Entities,mnb_label,lsvc_label,rf_label,mnb_prob,lsvc_prob,rf_prob
0,please cancel my order.,please cancel my order .,[],"[0.02432078184600029, 0.05269723870754585, 0.0...",Order Cancel,Order Cancel,0.758637,0.896239,0.583046
1,There is a crack on the screen,there be a crack on the screen,[],"[0.7929831222903111, 0.03675355663235511, 0.00...",EXG/SHO,EXG/SHO,0.792983,0.814739,0.303456
2,"Hi, The order says delivered, but i did not ...","hi , the order say deliver , but i do not rece...",[],"[0.15887723379481494, 0.05727687123554762, 0.0...",Online Status,EXG/SHO,0.556285,0.541149,0.191582
3,This order has never been received. Could you...,this order have never be receive . could you p...,[],"[0.029907399652413114, 0.03784184635032739, 0....",Online Status,Logistics Issues,0.756280,0.523144,0.449106
4,It has said that the mouse pad that I ordered ...,it have say that the mouse pad that i order be...,[DATE],"[0.06856231599116698, 0.05277145374935914, 0.0...",Online Return,Refund Status,0.537357,0.494124,0.275847
5,Hi I would just like to cancel my order i plac...,hi i would just like to cancel my order i plac...,[PERSON],"[0.033031178443371466, 0.04633771419567532, 0....",Order Cancel,Order Cancel,0.692951,0.708391,0.599239
6,"When I ordered this, I thought it was a power ...","when i order this , i think it be a power bank...",[],"[0.3409442851071274, 0.13721142038738085, 0.01...",EXG/SHO,EXG/SHO,0.340944,0.417007,0.259576
7,"I have cancelled the order, why is the money n...","i have cancel the order , why be the money not...",[],"[0.04106513906070287, 0.06832696178499233, 0.0...",Order Cancel,Order Cancel,0.336678,0.467297,0.376884
8,Hi - I would like to return the : Dell Premier...,hi - i would like to return the : dell premier...,"[PERSON, ORG, PERSON, CARDINAL]","[0.32491051784598357, 0.08089368900040625, 0.0...",Online Return,Online Return,0.431411,0.730288,0.392860
9,I requested a return and have not received an ...,i request a return and have not receive an ema...,[],"[0.02002232449385199, 0.026328523266873776, 0....",Online Return,Refund Status,0.475199,0.634886,0.443377


In [146]:
ResultsDf.shape

(18, 9)

In [147]:
# ResultsDf[['mnb_label','lsvc_label','rf_label']].mode(axis=1)

In [148]:

for sentence in corpus:
    displacy.render(nlp(sentence), jupyter=True, style='ent')

C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


C:\Users\raghu_chandra_katike\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [149]:
sentence = """hi, i asked to cancel this order. the order was not successfully delivered and was then returned. i checked the shipment information and found out that it has already been returned and signed by b.wolfe 10/28/2019 at 10:26 am. i was told that the refund would be processed within 10-15 business days after the return. a month passed, but i have not received the refund. could you please help me with that? thank you in advance!"""
displacy.render(nlp(sentence), jupyter=True, style='ent')


In [150]:
# Plan for Implementation

# - Read the comment
# - Perform text preprocessing,create vectors, send it through our model.
# - Analyze model output
#  - Check for prediction & probability
#  - Check for Sentiment
#  - Check for Length
# - Analyze the comment to see for entities? if there are multiple asks?
#  - What combination would we want to look at?
#     # Date # Money # MultipleOrgs # Time  -> All of these are atrributed for shipments delays
#     ??? what combinations to tap in to minimize False Positives
# - If a req is >80% probability & Pos Sentiment & cap on length & doesn't have multiple asks  == a request to get through for RPA to handle


# How can i use topic modelling to pull out these different attributes and combinations for shipment delays or logistic issues etc..??


